<a href="https://colab.research.google.com/github/nikyan/Covid-19_Research/blob/master/Covid_19_Research_Paper_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reaseach Paper Data ETL


The research paper data is obtained from Allen Intitute's Covid-19 open research data.
- https://www.semanticscholar.org/cord19/download
---



## Import libraries and load datasets.¶

- Load pdf.json into a dataframe and inspect the first few lines.
- Load pmc.json into a dataframe and inspect the first few lines.

In [0]:
# import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


%matplotlib inline

## Read and assess data

In [0]:
### 
#   This cell loads a function that prints the first n lines of
#   a file.
#
#   Then this function is called on the JSON file to print out
#   the first line of one of the research paper's json file
###

def print_lines(n, file_name):
    f = open(file_name)
    for i in range(n):
        print(f.readline())
    f.close()

In [7]:
# print first 20 lines of one json file to see the file structure
print_lines(20, '/content/drive/My Drive/ML data/AI2 PDFs/0036b28fddf7e93da0970303672934ea2f9944e7.json')

{

    "paper_id": "0036b28fddf7e93da0970303672934ea2f9944e7",

    "metadata": {

        "title": "The fecal microbiota and unconjugated fecal bile acids in dogs with diabetes mellitus ESCG-O-2 Impact of antibiotic administration on fecal bacterial groups potentially associated with dysbiosis in kittens ESCG-O-3 Fecal microbial metabolism is altered in dogs with chronic enteropathy ESCG-O-4 The pug breed demonstrates a worse response to treatment of protein-losing enteropathy than other breeds of dog ESCG-O-6 Dogs with acute haemorrhagic diarrhoea syndrome not receiving antibiotics have a good prognosis despite initial high AHDS-score and systemic inflammation ESCG-O-7 Faecal bile acid profiles in dogs with acute haemorrhagic diarrhoea syndrome over time and compared to healthy dogs ESCG-O-8 Long-term consequences of acute hemorrhagic diarrhea syndrome in dogs ESVC-O-1 Acute effect of oral pimobendan on left atrial function and mitral valve regurgitation severity in dogs with stage B

## 3. Extract and Load data to a Dataframe

In [11]:
# get list of all files in directory
directory_in_str = '/content/drive/My Drive/ML data/AI2 PDFs/'
directory = os.fsencode(directory_in_str)

#get filenames
filenames = []
for file in os.listdir(directory):
    filenames.append(os.fsdecode(file))

print("Number of files to read: {}".format(len(filenames)))

Number of files to read: 4776


In [0]:
# loop through each file to extract data: {paper id, title, body text}
d1 = []
for file in filenames:
    # read in the JSON file
    with open('/content/drive/My Drive/ML data/AI2 PDFs/{}'.format(file)) as f:
        json_data = json.load(f)
    
    
    #paper_id = {'paper_id': json_data['paper_id']}
    #bodytext = {'body_text': json_data['body_text']}
    text = ""
    for i in range(len(json_data['body_text'])):
        t = json_data['body_text'][i]['text']
        text = text + t
    
    t_d1 = {'paper_id': json_data['paper_id'], 'title': json_data['metadata']['title'],'text': text}
    
    d1.append(t_d1)

In [13]:
# create a dataframe from extracted data
df = pd.DataFrame(d1)
print(df.shape)
df.head(10)

(4776, 3)


,paper_id,title,text
0,5cf53c279192e23b92dd29e74cf9bdbb3b598244,Assessment of transport stress on cattle trave...,The effect of long-distance transport on cattl...
1,4ec7c4ec545371f4b8c4a40db7d28fc534674c39,"Knowledge, Perceptions, and Self-reported Perf...",Health care-associated infections (HAIs) criti...
2,06f075e64c8861491ea4de2f98b606866f54a752,case report,M iddle east respiratory syndrome coronavirus ...
3,40c0532640a8bb9d9a0aac07d017d5982af33fdb,Mutation of the S and 3c genes in genomes of f...,same owner and referred to a veterinary hospit...
4,0e18097a83b2801c4865c72065b7933fa24face3,Nonproliferative and Proliferative Lesions of ...,The INHAND Project (International Harmonizatio...
5,83e5553035f500f4bcc4aeeaa41aa74ebf578567,A Single-Center Study of Viral Respiratory Tra...,Viral respiratory infections are among the mos...
6,47bd7df352db2b1b093566b04fc328d6f0292c1b,Comparison of fit factors among healthcare pro...,Healthcare providers are at risk of exposure t...
7,15d87194ff22d1a52809b2c2332aa3e11a6fcd4a,The Journal of Infectious Diseases High Enviro...,Hepatitis B virus (HBV) infection is a severe ...
8,5ab92ddf1bb39b6863b28e2e9946415f7a5adf34,A survey on infection control in emergency dep...,E MERGENCY DEPARTMENTS ARE the entrance to hos...
9,4fab9d6fcfd7daca9ed2e0f89c2433955d34e553,TITLE (PROVISIONAL) Geographical disparities i...,to specifically identify the presentation of t...


In [0]:
# save dataframe to a sqlite db.
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../content/drive/My Drive/ML data/ai2_research_data.db')
df.to_sql('research_data', engine, index=False, if_exists='replace')